In [0]:
df = spark.read.format("json")\
    .option("inferSchema", True)\
    .option("multiline",True)\
    .load("/Volumes/pyspark_cata/source/db_volume/jsonData/")

display(df)

customer,delivery_updates,items,order_id,order_timestamp,payment
"List(CUST101, john.doe@example.com, List(Toronto, Canada), John Doe)","List(Order Placed, Packed, Shipped, Out for Delivery)","List(List(ITEM1001, 25.5, Wireless Mouse, 2), List(ITEM1002, 199.75, Mechanical Keyboard, 1))",ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)"
"List(CUST102, jane.smith@example.com, List(Vancouver, Canada), Jane Smith)","List(Order Placed, Packed, Shipped)","List(List(ITEM1003, 89.99, USB-C Hub, 1))",ORD002,2025-08-15T11:10:15Z,"List(89.99, CAD, PayPal)"


In [0]:
df.schema

StructType([StructField('customer', StructType([StructField('customer_id', StringType(), True), StructField('email', StringType(), True), StructField('location', StructType([StructField('city', StringType(), True), StructField('country', StringType(), True)]), True), StructField('name', StringType(), True)]), True), StructField('delivery_updates', ArrayType(StringType(), True), True), StructField('items', ArrayType(StructType([StructField('item_id', StringType(), True), StructField('price_per_unit', DoubleType(), True), StructField('product_name', StringType(), True), StructField('quantity', LongType(), True)]), True), True), StructField('order_id', StringType(), True), StructField('order_timestamp', StringType(), True), StructField('payment', StructType([StructField('amount', DoubleType(), True), StructField('currency', StringType(), True), StructField('method', StringType(), True)]), True)])

In [0]:
from pyspark.sql.functions import *

In [0]:
df_cust = df.select("customer.customer_id", "customer.email", "customer.location.country","*").drop("customer")
 #when it is not dictionary 
df_cust_upd= df_cust.withColumn("delivery_updates", explode("delivery_updates"))\
    .withColumn("items", explode("items"))\
    .select("*", "items.*").drop("items")\
    .select("*", "payment.*")

display(df_cust_upd)


customer_id,email,country,delivery_updates,order_id,order_timestamp,payment,item_id,price_per_unit,product_name,quantity,amount,currency,method
CUST101,john.doe@example.com,Canada,Order Placed,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1001,25.5,Wireless Mouse,2,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Order Placed,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1002,199.75,Mechanical Keyboard,1,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Packed,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1001,25.5,Wireless Mouse,2,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Packed,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1002,199.75,Mechanical Keyboard,1,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Shipped,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1001,25.5,Wireless Mouse,2,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Shipped,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1002,199.75,Mechanical Keyboard,1,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Out for Delivery,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1001,25.5,Wireless Mouse,2,250.75,CAD,Credit Card
CUST101,john.doe@example.com,Canada,Out for Delivery,ORD001,2025-08-15T10:45:30Z,"List(250.75, CAD, Credit Card)",ITEM1002,199.75,Mechanical Keyboard,1,250.75,CAD,Credit Card
CUST102,jane.smith@example.com,Canada,Order Placed,ORD002,2025-08-15T11:10:15Z,"List(89.99, CAD, PayPal)",ITEM1003,89.99,USB-C Hub,1,89.99,CAD,PayPal
CUST102,jane.smith@example.com,Canada,Packed,ORD002,2025-08-15T11:10:15Z,"List(89.99, CAD, PayPal)",ITEM1003,89.99,USB-C Hub,1,89.99,CAD,PayPal


In [0]:
df_cust.printSchema()


root
 |-- customer_id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- delivery_updates: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- price_per_unit: double (nullable = true)
 |    |    |-- product_name: string (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_timestamp: string (nullable = true)
 |-- payment: struct (nullable = true)
 |    |-- amount: double (nullable = true)
 |    |-- currency: string (nullable = true)
 |    |-- method: string (nullable = true)

